In [ ]:
import numpy as np
import importlib, os, datetime, pickle
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from quick_sim import setup_sim

# import edward_tools.fq_runner as fq_runner
from edward_tools.simple_1D_potential import simple_1D_potential, simple_1D_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from NAND_PARAMETERS import *
import importlib
import kyle_tools as kt
import matplotlib.pyplot as plt

from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization
from edward_tools.potential_runner_1D import simRunner

create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
# parameter settings

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 1
I_m_factor = 0
time_scale = 1.0

I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F
L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)

In [ ]:
m_c, m_1, m_2 = C_1, C_1, C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
kappa_1 = 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

_lambda = lambda_1
_theta  = theta_1
_eta    = eta_1
v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c

In [ ]:
init_state_used = None
phi_1_dcx, phi_2_dcx = 0, 0
has_velocity = True
saveAllStates = False

params = {}
params['N'] = 10
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "test"
params['capacitance'] = C_1
params['mass'] = 1
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::10] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state

In [ ]:
import numpy as np
init_state = np.array([0.5, 1])

In [ ]:
def linear_interpolation(y_i, y_f, t_i, t_f):
    def func(t):
        return (y_f - y_i) / (t_f - t_i) * t + y_i
    return func

In [ ]:
def get_params(protocol_list, interpolation_method, t):
    protocol_list_np_array = np.array(list(protocol_list.values()))
    protocol_key = np.array(list(protocol_list.keys()))
    time_array = protocol_list_np_array[0]
    parameter_array = protocol_list_np_array[1:]
    index_i = np.sum(time_array < t) - 1
    index_f = index_i + 1
    t_i, t_f = time_array[index_i], time_array[index_f]
    # print(index_i, index_f, t_i, t_f)
    interpolate_value = interpolation_method(parameter_array[:, index_i], parameter_array[:, index_f], 0, t_f - t_i)(t - t_i)
    return interpolate_value

In [ ]:
get_params(protocol_list, linear_interpolation, 0)

In [ ]:
def simple_1D_potential(x, params):
    m, c = params
    return 1/2 * m * x**2

def simple_1D_force(x, params):
    m, c = params
    return - m * x

def simple_line_potential(x, params):
    m, c = params
    return m * x + c

def simple_line_force(x, params):
    m, c = params
    return -m

def cubic_potential(x, params):
    m, c = params
    return m * (x - c)**5

def cubic_force(x, params):
    m, c = params
    return - 5 * m * (x - c)**4


def qfp_potential(phi_1, params):
    beta, gamma, phi_1x, phi_2x, phi_1xdc, phi_2xdc, mu_12 = params 
    u_1 = 1/2 * (phi_1 - phi_1x)**2 + beta * np.cos(phi_1) * np.cos(phi_1xdc/2)
    u_12 = mu_12 * (phi_1 - phi_1x) * (2 - phi_2x)
    return u_1 + u_12

def qfp_force(phi_1, params):
    beta, gamma, phi_1x, phi_2x, phi_1xdc, phi_2xdc, mu_12 = params 
    du_1 = (phi_1 - phi_1x)**2 - beta * np.sin(phi_1) * np.cos(phi_1xdc/2)
    du_12 = mu_12 * (2 - phi_2x)
    return -(du_1 + du_12)
    
def get_KE(state):
    x = state[..., 0]
    v = state[..., 1]
    return 1/2 * v**2

In [ ]:
def updateMethod(langevinUnderdamped_object):
    def get_determ_dsdt(state, time):
        theta = langevinUnderdamped_object['theta']
        gamma = langevinUnderdamped_object['gamma']
        get_external_force = langevinUnderdamped_object['get_external_force']
        params_at_t = get_params(langevinUnderdamped_object['protocol_list'], langevinUnderdamped_object['interpolation_method'], time)

        position = state[..., 0]
        velocity = state[..., 1]
        
        determ_dxdt = velocity
        determ_dvdt = theta * get_external_force(position, params_at_t) - gamma * velocity
        determ_dsdt = np.empty(state.shape)
        determ_dsdt[..., 0] = determ_dxdt
        determ_dsdt[..., 1] = determ_dvdt

        return determ_dsdt


    def get_stoch_dsdt(self, state, time, rng):

        eta = self.eta

        # want random numbers for only the velocity values
        vel_rng_shape = state.shape[:-1]

        stoch_dxdt = 0
        stoch_dvdt = eta * sqrt(2) * rng.normal(size=vel_rng_shape)

        stoch_dsdt = empty(state.shape)
        stoch_dsdt[..., 0] = stoch_dxdt
        stoch_dsdt[..., 1] = stoch_dvdt

        return stoch_dsdt


    def update_state(self, state, time, dt):
        # get_determ_dsdt = self.get_determ_dsdt
        # get_stoch_dsdt = self.get_stoch_dsdt
        # rng = self.rng
        k1 = get_determ_dsdt(state,               time)
        k2 = get_determ_dsdt(state + k1 * dt / 2, time + dt / 2)
        k3 = get_determ_dsdt(state + k2 * dt / 2, time + dt / 2)
        k4 = get_determ_dsdt(state + k3 * dt,     time + dt)

        ds_determ = dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

        ds_stoch = 0
        # ds_stoch = get_stoch_dsdt(state, time, rng) * sqrt(dt)

        next_state = state + ds_determ + ds_stoch

        return next_state
    
    return get_determ_dsdt, get_stoch_dsdt, update_state

In [ ]:
t =  np.linspace(0, 10, 101),
m = np.array([1, -1] * 25 + [1, -0.8] * 26 )[0:101],
c = [0] * 50 + [-1] * 51

# protocol

In [ ]:
weird_m = np.empty(101)

In [ ]:
scale_array = np.linspace(1, 1.5, 25)

In [ ]:
scale_array

In [ ]:
weird_m = np.array([1, -1] * 25 + [-2] + [1, -1] * 25)[0:101]

In [ ]:
protocol_list

In [ ]:
protocol_list = {
    "t": np.array([0, 2, 5]),
    "m": np.array([1, 2, 3]),
    "c": np.array([0, 0, 0])
}

protocol_list = {
    "t": np.linspace(0, 5, 101),
    "m":  weird_m,
    "c": np.array([-0.5] * 50 + [-0.5] * 51)
}

protocol_list = {
    "t": np.linspace(0, 10, 11),
    "beta": np.full(11, 2.3),
    "gamma": np.full(11, 10),
    "phi_1x": np.linspace(0, 0.61, 11),
    "phi_2x": np.linspace(0, 0.1, 11),
    "phi_1xdc": np.linspace(0, 1.79, 11),
    "phi_2xdc": np.full(11, 0),
    "mu_12": np.linspace(0, 0.06, 11)
}

In [ ]:
potential_used = qfp_potential
force_used = qfp_force
langevinUnderdamped_object = { 
    "theta": 1, "gamma": 0, "lambda": 0,
    "get_potential":        potential_used,
    "get_external_force":   force_used,
    "protocol_list":        protocol_list,
    "interpolation_method": linear_interpolation
}

In [ ]:
get_determ_dsdt, get_stoch_dsdt, update_state = updateMethod(langevinUnderdamped_object)

In [ ]:
# init_state = np.array([[1, -0.2], [-1, 0.2]])
t_total = protocol_list['t'][-1]
N_particle = 100
N_step = int(t_total / dt)
dt = 0.01
current_step  = 0
current_time = 0


"""generate initial state"""
init_state = np.empty([N_particle, 2])
# particle_00 
init_state[0:int(N_particle/2), 0] = np.random.normal(2, 0.1, int(N_particle/2))
init_state[0:int(N_particle/2), 1] = np.random.normal(0, 0.1, int(N_particle/2))

# particle_01 
init_state[int(N_particle/2):, 0] = np.random.normal(-2, 0.1, int(N_particle/2))
init_state[int(N_particle/2):, 1] = np.random.normal(0, 0.1, int(N_particle/2))

particle_index_00 = init_state[..., 0] < 0
particle_index_01 = init_state[..., 0] > 0

"""set_up variables to store datas at all the states"""
current_state = np.empty(init_state.shape)

all_states = np.zeros([N_step + 1, init_state.shape[0], init_state.shape[1]])
all_time = np.zeros(N_step + 1)
all_potential = np.zeros([N_step + 1, init_state.shape[0]])
work_done = np.zeros([N_step, init_state.shape[0]])

params_t_0 = get_params(protocol_list, linear_interpolation, 0)

all_time[0] = current_time
all_states[0, ...] = init_state
all_potential[0, ...] = potential_used(init_state[..., 0], params_t_0)

In [ ]:
for _ in range(0, N_step-1):
    current_time += dt
    current_step += 1

    current_params = get_params(protocol_list, linear_interpolation, current_time)

    # calculate potential of the previous state at the current time to get the work done
    all_potential[current_step] = potential_used(all_states[current_step-1][..., 0], current_params)
    
    # update time and states
    all_time[current_step] = current_time
    all_states[current_step, ...] = update_state(langevinUnderdamped_object, all_states[current_step-1], current_time, dt)

In [ ]:
# print(all_time[:current_step+1])
# print(all_states[:current_step+1, ...])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(all_states[:current_step+1, :, 0], all_potential[:current_step+1])

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter

In [ ]:
potential_used

In [ ]:
plt.plot(get_KE(all_states))
plt.show()

In [ ]:
frame_skip = 10
selected_all_states = all_states[::frame_skip, ...]
selected_time = all_time[::frame_skip]

x_array = np.linspace(-2, 2, 100)
potential_at_0 = potential_used(x, params_t_0)
fig, ax = plt.subplots(1, 1, figsize = [10, 5])
potential_plot = ax.plot(x, potential_at_0)[0]
# scatter_plot = plt.scatter(init_state[..., 0], all_potential[0, ...])
scatter_plot_00 = plt.scatter(init_state[particle_index_00, 0], all_potential[0, particle_index_00], color = "red")
scatter_plot_01 = plt.scatter(init_state[particle_index_01, 0], all_potential[0, particle_index_01], color = "blue")


def animate(i):
    current_t = selected_time[i]
    current_state = selected_all_states[i]
    current_params = get_params(protocol_list, linear_interpolation, current_t)
    
    ax.set_title(f"t = {current_t:.3g}")
    
    potential_at_t = potential_used(x, current_params)
    particle_potential_at_t = potential_used(current_state[..., 0], current_params)
    
    # scatter_plot.set_offsets(np.c_[current_state[..., 0], particle_potential_at_t])
    scatter_plot_00.set_offsets(np.c_[current_state[particle_index_00, 0], particle_potential_at_t[particle_index_00]])
    scatter_plot_01.set_offsets(np.c_[current_state[particle_index_01, 0], particle_potential_at_t[particle_index_01]])
    
    potential_plot.set_ydata(potential_at_t)
    
    
ani = FuncAnimation(fig, animate, interval=100, frames=len(selected_time))
writer = PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()

In [ ]:
from IPython.display import HTML, Image
Image(filename="scatter.gif")

In [ ]:
# work_analysis